In [1]:
%load_ext autoreload
%autoreload 1
%aimport treeflow
%aimport treeflow.evolution.seqio
%aimport treeflow.evolution.calibration.calibration

2022-04-26 21:51:38.057891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-26 21:51:38.057916: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Treeflow example - carnivores data

Dataset from **Suchard, Marc A., and Andrew Rambaut. "Many-core algorithms for statistical phylogenetics." *Bioinformatics* 25.11 (2009): 1370-1376.**

Sequences accessed from [Beast examples on Github](https://github.com/beast-dev/beast-mcmc/blob/v1.10.4/examples/Benchmarks/benchmark2.xml).

In [2]:
import treeflow

## Read input data

In [3]:
tree = treeflow.parse_newick("demo-data/carnivores.newick")
tree.taxon_set

('Puma_concolor_', 'Acinonyx_jubatus_', 'Lynx_canadensis_', 'Felis_silvestris_', 'Uncia_uncia_', 'Panthera_pardus_', 'Panthera_tigris_', 'Neofelis_nebulosa_', 'Herpestes_auropunctatus_', 'Leptonychotes_weddellii_', 'Hydrurga_leptonyx_', 'Ommatophoca_rossii_', 'Lobodon_carcinophaga_', 'Mirounga_leonina_', 'Mirounga_angustirostris_', 'Monachus_monachus_', 'Monachus_schauinslandi_', 'Phoca_fasciata_', 'Phoca_groenlandica_', 'Phoca_caspica_', 'Halichoerus_grypus_', 'Phoca_sibirica_', 'Phoca_largha_', 'Phoca_vitulina_', 'Phoca_hispida_', 'Cystophora_cristata_', 'Erignathus_barbatus_', 'Arctocephalus_forsteri_', 'Arctocephalus_australis_', 'Arctocephalus_townsendi_', 'Neophoca_cinerea_', 'Phocarctos_hookeri_', 'Arctocephalus_pusillus_', 'Otaria_byronia_', 'Zalophus_californianus_', 'Eumetopias_jubatus_', 'Callorhinus_ursinus_', 'Odobenus_rosmarus_', 'Enhydra_lutris_', 'Lontra_canadensis_', 'Mustela_vison_', 'Martes_melampus_', 'Martes_americana_', 'Gulo_gulo_', 'Meles_meles_', 'Taxidea_taxus

In [4]:
alignment = treeflow.Alignment("demo-data/carnivores.fasta").get_compressed_alignment()
alignment

WeightedAlignment(taxon_count=62, pattern_count=5565)

Calibrations from **Hassanin, Alexandre, et al. "Evolutionary history of Carnivora (Mammalia, Laurasiatheria) inferred from mitochondrial genomes." *PloS one* 16.2 (2021): e0240770.**

In [5]:
import yaml
with open("demo-data/carnivore-calibrations.yaml") as f:
    calibration_dict = yaml.safe_load(f)
    
calibration_set = treeflow.MRCACalibrationSet(calibration_dict)
calibration_set

MRCACalibrationSet(MRCACalibration(name='Guloninae, Mustilinae', range=(12.2, 27.6)), MRCACalibration(name='Felidae', range=(14, 20)), MRCACalibration(name='Pinnipedia', range=(19, 34)))

## Model specification

### No site rate variation

In [6]:
import tensorflow_probability as tfp
from treeflow import float_constant as c

base_model = tfp.distributions.JointDistributionNamed(dict(
    birth_rate=tfp.distributions.LogNormal(c(1.0), c(1.5)),
    tree=lambda birth_rate: treeflow.distributions.Yule(tree.taxon_count, birth_rate)
))
base_model.sample()

2022-04-26 21:51:41.528483: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-26 21:51:41.528538: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-26 21:51:41.528563: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CS424460): /proc/driver/nvidia/version does not exist
2022-04-26 21:51:41.529032: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/cswa648/dev/treeflow/treeflow/distributions/tree/birthdeath/birth_death_contemporary_sampling.py:66: UserWar

{'birth_rate': <tf.Tensor: shape=(), dtype=float64, numpy=22.505263477593754>,
 'tree': TensorflowRootedTree(node_heights=<tf.Tensor: shape=(61,), dtype=float64, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])>, sampling_times=<tf.Tensor: shape=(62,), dtype=float64, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])>, topology=TensorflowTreeTopology(parent_indices=<tf.Tensor: shape=(122,), dtype=int32, numpy=
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Results visualisation


* Use existing tree viz tools